In [20]:
import pandas as pd
import pymongo
import numpy as np
from geopy.geocoders import Nominatim

pd.set_option("display.max_rows", 300)

### Extract and Clean Data

#### Population of Cities

In [21]:
# url to scrape for the city population
cities_url ="https://worldpopulationreview.com/world-cities"
# Use pandas `read_html` to parse the url
df_cityPop = pd.read_html(cities_url, header=0)[0]
# rename the columns
df_cityPop.rename(columns={'Name':'City', 
                           '2020 Population':'2020',
                           '2019 Population':'2019'
                          },inplace=True)
# Replace null values with 0
df_cityPop.fillna(0,inplace = True)
#  converting column 2019 from float to int
df_cityPop['2019'] = df_cityPop['2019'].apply(np.int32)

df_cityPop.head()

,Rank,City,Country,2020,2019,Change
0,1,Tokyo,Japan,37393128,37435192,-0.11%
1,2,Delhi,India,30290936,29399140,3.03%
2,3,Shanghai,China,27058480,26317104,2.82%
3,4,Sao Paulo,Brazil,22043028,21846508,0.90%
4,5,Mexico City,Mexico,21782378,21671908,0.51%


In [22]:
# Using geopy for coordinates of top 10 cities
# creating a dataframe with coordinates
cities = []
# iterate through top 10 rows
for row in df_cityPop.head(10).itertuples():
    try:
        geolocator = Nominatim(user_agent="population_analysis")
        city = row[2]
        country = row[3]
        loc = geolocator.geocode(city+','+ country)
        
        cities.append({"City": city,
                    "Country": country,
                    "Latitude": loc.latitude, 
                    "Longitude": loc.longitude}) 
    except:
        print("City not found. Skipping...") 

city_df = pd.DataFrame(cities)
city_df.head()

,City,Country,Latitude,Longitude
0,Tokyo,Japan,35.682839,139.759455
1,Delhi,India,28.651718,77.221939
2,Shanghai,China,31.232276,121.469207
3,Sao Paulo,Brazil,-23.550651,-46.633382
4,Mexico City,Mexico,19.432630,-99.133178


In [23]:
# merging city dataframes 
df_cityPop = df_cityPop.merge(city_df, on=["City","Country"], how="left")
df_cityPop.head()

,Rank,City,Country,2020,2019,Change,Latitude,Longitude
0,1,Tokyo,Japan,37393128,37435192,-0.11%,35.682839,139.759455
1,2,Delhi,India,30290936,29399140,3.03%,28.651718,77.221939
2,3,Shanghai,China,27058480,26317104,2.82%,31.232276,121.469207
3,4,Sao Paulo,Brazil,22043028,21846508,0.90%,-23.550651,-46.633382
4,5,Mexico City,Mexico,21782378,21671908,0.51%,19.432630,-99.133178


#### Scraping Country codes to merge datasets with

In [24]:
# url to scrape for ISO 3166 country codes Alpha-2 and Alpha-3 from www.iban.com
country_code_url ="https://www.iban.com/country-codes"
# Use panda's `read_html` to parse the url
df_countryCode = pd.read_html(country_code_url, header=0)[0]
# eliminating unnessasary data
df_countryCode = df_countryCode.iloc[:,[1,2]]
# rename the columns
df_countryCode.rename(columns={'Alpha-2 code':'Country_Code_2',
                               'Alpha-3 code':'Country_Code'
                              },inplace=True)


df_countryCode.head()

,Country_Code_2,Country_Code
0,AF,AFG
1,AX,ALA
2,AL,ALB
3,DZ,DZA
4,AS,ASM


#### Population of Countries

In [25]:
# read Countries population data from csv(source:https://worldpopulationreview.com) into dataframe
df_countries = pd.read_csv('static/data/countriesData.csv')
# rename the columns
df_countries.rename(columns={'cca2':'Country_Code_2',
                             'name':'Country',
                             'pop2050':'2050',
                             'pop2030':'2030',
                             'pop2020':'2020',
                             'pop1980':'1980',
                             'pop1970':'1970'
                            },inplace=True)

# eliminating flag column and rerodering
df_countries = df_countries.iloc[:,[0,1,4,5,2,10,11,12,13,14,15,16]]

# Removing decimal point from data
col_list = ['2050','2030','2020','1980','1970']
# Loop through the columns
for col in df_countries[col_list]:
    # performing operations on columns other than Country column
    if col not in ["Country_Code_2", "Country"]:
        # correcting the decimal positions
        df_countries[col] = (df_countries[col] * 1000).apply(np.int64)

df_countries.head()

,Country_Code_2,Country,2050,2030,2020,1980,1970,area,Density,GrowthRate,WorldPercentage,rank
0,CN,China,1402405170,1464340159,1439323776,1000089235,827601394,9706961,148.2775,1.0039,0.1847,1
1,IN,India,1639176033,1503642322,1380004385,698952844,555189792,3287590,419.7617,1.0099,0.1770,2
2,US,United States,379419102,349641876,331002651,229476354,209513341,9372610,35.3160,1.0059,0.0425,3
3,ID,Indonesia,330904664,299198430,273523615,147447836,114793178,1904569,143.6144,1.0107,0.0351,4
4,PK,Pakistan,338013196,262958794,220892340,78054343,58142060,881912,250.4698,1.0200,0.0283,5


In [26]:
# Another Dataset
# Cleaning csv Population data from https://datacatalog.worldbank.org
# reading csv's into dataframes
df_population = pd.read_csv('static/data/population.csv')


# Function to Clean each dataframes
def clean_dataFrames(df, col_list):
    # eliminating unnecessary data
    df = df.iloc[0:217, col_list]
    # renaming columns
    df.rename(columns= {df.columns[0]: "Name"}, inplace = True)
    df = df.rename(columns = lambda x : (str(x)[:-9]))
    df.rename(columns= {df.columns[0]: "Country", df.columns[1]: "Country_Code"}, inplace = True)
    return df

# list of required column indexes
col_list = [2,3,4,5,10,11,12,13,14]
# Calling clean_dataFrames function passing the dataframe as parameter
df_population = clean_dataFrames(df_population, col_list)

# replacing string ".." value in data
df_population = df_population.replace("..", 0)

# Loop through the columns to covert values from string to 
for col in df_population:
    # on columns other than Country and Country_Code_2
    if col not in ["Country_Code", "Country"]:
        # Converting string to number
        df_population[col] = df_population[col].astype(float).apply(np.int64)


df_population.tail()

,Country,Country_Code,1990,2000,2015,2016,2017,2018,2019
212,Virgin Islands (U.S.),VIR,103963,108642,107710,107510,107268,106977,106631
213,West Bank and Gaza,PSE,1978248,2922153,4270092,4367088,4454805,4569087,4685306
214,"Yemen, Rep.",YEM,11709993,17409072,26497889,27168210,27834821,28498687,29161922
215,Zambia,ZMB,8036845,10415944,15879361,16363507,16853688,17351822,17861030
216,Zimbabwe,ZWE,10432421,11881477,13814629,14030390,14236745,14439018,14645468


In [27]:
# merging two dataframes for additional years data
# merging df_population with df_countryCode
df_population = df_countryCode.merge(df_population, on="Country_Code", how="inner")

# merging df_population with df_countries
df_countries_merged = df_countries.merge(df_population, on="Country_Code_2", how="inner")

# removing duplicated Country column and Country_Code_2
df_countries_merged = df_countries_merged.drop(['Country_y', 'Country_Code_2'], axis=1)
# renaming columns
df_countries_merged.rename(columns= {"Country_x": "Country"}, inplace = True)
# reordering the columns
df_countries_merged = df_countries_merged.iloc[:,[0,11,1,2,3,18,17,16,15,14,13,12,4,5,6,7,8,9,10]]
# Replace null values with 0
df_countries_merged.fillna(0,inplace = True)

# Keeping a dataframe of Countries and country codes
df_countries = df_countries_merged.iloc[:,[0,1]]

df_countries_merged.head()

,Country,Country_Code,2050,2030,2020,2019,2018,2017,2016,2015,2000,1990,1980,1970,area,Density,GrowthRate,WorldPercentage,rank
0,China,CHN,1402405170,1464340159,1439323776,1397715000,1392730000,1386395000,1378665000,1371220000,1262645000,1135185000,1000089235,827601394,9706961,148.2775,1.0039,0.1847,1
1,India,IND,1639176033,1503642322,1380004385,1366417754,1352617328,1338658835,1324509589,1310152403,1056575549,873277798,698952844,555189792,3287590,419.7617,1.0099,0.1770,2
2,United States,USA,379419102,349641876,331002651,328239523,326687501,324985539,322941311,320635163,282162411,249623000,229476354,209513341,9372610,35.3160,1.0059,0.0425,3
3,Indonesia,IDN,330904664,299198430,273523615,270625568,267663435,264645886,261554226,258383256,211513823,181413402,147447836,114793178,1904569,143.6144,1.0107,0.0351,4
4,Pakistan,PAK,338013196,262958794,220892340,216565318,212215030,207896686,203627284,199426964,142343578,107647921,78054343,58142060,881912,250.4698,1.0200,0.0283,5


#### World Population and Projections

In [28]:
# read World population data from csv(source:https://worldpopulationreview.com) into dataframe
df_worldPop = pd.read_csv('static/data/worldPopulation.csv')
# Sort in ascending order
df_worldPop.sort_values(by=["year"], inplace=True)
# Projections
df_worldPojection = pd.read_csv('static/data/worldProjections.csv')

# Concatenate both actuals and projections dataframe
df_worldPop = pd.concat([df_worldPop,df_worldPojection])
# rename the columns
df_worldPop.rename(columns={'value':'Population'},inplace=True)
# Replace null values with 0
df_worldPop.fillna(0,inplace = True)

df_worldPop.head()

,year,Population,GrowthRate
14,1950,2536431149,0.0000
13,1955,2773019936,0.0180
12,1960,3034949748,0.0182
11,1965,3339583597,0.0193
10,1970,3700437046,0.0207


#### Population Density

In [34]:
# Defining a function to clean similar dataframes
def cleanData(df, col_list, subset):
    #  eliminating unnecessary data
    df = df.iloc[:, col_list]
    # renaming columns
    df.rename(columns= {"Country Name":"Country", "Country Code":"Country_Code"}, inplace = True)
    # merging with countries_df to keep the Countries List same
    df = df_countries.merge(df, on=["Country_Code"], how="inner")
    
    df = df.drop(['Country_y'], axis=1)
    df.rename(columns= {"Country_x":"Country"}, inplace = True)
    # removing countries with NaN value in 2017
    df = df.dropna(axis=0, subset=subset)
    # Replace null values with 0
    df.fillna(0,inplace = True)
    return df


# read World population Density data from https://datacatalog.worldbank.org into dataframe
df_Density = pd.read_csv('static/data/populationDensity.csv')
#  making list of necessary data
col_list = [0,1,5,14,24,34,44,54,59,60,61,62]
# Calling cleanData function with dataframe and column lists as parameters
df_Density =cleanData(df_Density, col_list, ['2017'])

df_Density.head()

,Country,Country_Code,1961,1970,1980,1990,2000,2010,2015,2016,2017,2018
0,China,CHN,70.335792,87.163742,104.517349,120.915506,134.492481,142.487740,146.057642,146.850678,147.674051,148.348833
1,India,IND,154.595625,186.732026,235.085159,293.717454,355.367652,415.136998,440.655459,445.484341,450.243286,454.938073
2,United States,USA,20.055880,22.388131,24.809039,27.254514,30.797301,33.815181,35.051978,35.304087,35.527563,35.713622
3,Indonesia,IDN,49.734978,63.366681,81.392293,100.141536,116.757190,133.494270,142.629463,144.379862,146.086481,147.752190
4,Pakistan,PAK,59.756682,75.422971,101.253558,139.642903,184.650760,232.753011,258.700400,264.149134,269.687482,275.289319


#### Birth Rate, Mortality Rate and Sex Ratio

In [30]:
# Cleaning csv Birth rate data from https://datacatalog.worldbank.org
# reading csv's into dataframes
df_birth = pd.read_csv('static/data/birthRate.csv')

#  making list of necessary data
col_list = [0,1,4,14,24,34,44,54,59,60,61,62]
# Calling cleanData function with dataframe and column lists as parameters
df_birth =cleanData(df_birth, col_list, ['2018'])


df_birth.head()

,Country,Country_Code,1960,1970,1980,1990,2000,2010,2015,2016,2017,2018
0,China,CHN,20.860,33.430,18.210,21.060,14.030,11.900,12.070,12.950,12.430,10.900
1,India,IND,41.995,39.108,36.167,31.517,26.401,21.114,18.625,18.332,18.083,17.857
2,United States,USA,23.700,18.400,15.900,16.700,14.400,13.000,12.400,12.200,11.800,11.600
3,Indonesia,IDN,44.549,39.973,33.425,25.813,21.773,20.795,19.165,18.790,18.422,18.072
4,Pakistan,PAK,44.111,43.130,42.260,41.222,34.703,30.082,29.124,28.888,28.599,28.250


#### Mortality Rate

In [31]:
# Cleaning csv Death rate data from https://datacatalog.worldbank.org
# reading csv's into dataframes
df_death = pd.read_csv('static/data/deathRate.csv')
# Calling cleanData function with dataframe and column lists as parameters
df_death = cleanData(df_death, col_list, ['2018'])

df_death.head()

,Country,Country_Code,1960,1970,1980,1990,2000,2010,2015,2016,2017,2018
0,China,CHN,25.430,7.600,6.340,6.670,6.450,7.110,7.110,7.090,7.110,7.100
1,India,IND,22.184,17.187,13.290,10.864,8.692,7.492,7.194,7.195,7.210,7.234
2,United States,USA,9.500,9.500,8.800,8.600,8.500,7.995,8.440,8.493,8.638,8.600
3,Indonesia,IDN,19.177,14.308,10.592,8.339,7.475,6.653,6.419,6.418,6.433,6.465
4,Pakistan,PAK,20.675,15.294,12.670,10.828,8.862,7.568,7.107,7.048,6.994,6.942


#### Sex Ratio

In [32]:
# Cleaning csv sex Ratio data from https://datacatalog.worldbank.org
# reading csv's into dataframes
df_sexRatio = pd.read_csv('static/data/sexRatio.csv')
#  making list of necessary data
col_list = [0,1,6,16,26,36,46,54,59,60,61,62]
# Calling cleanData function with dataframe and column lists as parameters
df_sexRatio = cleanData(df_sexRatio, col_list, ['2018'])

df_sexRatio.head()

,Country,Country_Code,1962,1972,1982,1992,2002,2010,2015,2016,2017,2018
0,China,CHN,1.070,1.070,1.070,1.120,1.160,1.158,1.138,1.134,1.130,1.126
1,India,IND,1.054,1.057,1.084,1.106,1.112,1.100,1.099,1.099,1.099,1.100
2,United States,USA,1.050,1.053,1.051,1.049,1.048,1.048,1.047,1.047,1.047,1.047
3,Indonesia,IDN,1.050,1.050,1.050,1.050,1.050,1.050,1.050,1.050,1.050,1.050
4,Pakistan,PAK,1.064,1.064,1.064,1.064,1.096,1.087,1.087,1.087,1.087,1.087


### Load Data into MongoDB

In [33]:
# Loading Data into MongoDB
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

db_name = "populationDB"
# # Drop database if exists
if bool(db_name in client.list_database_names()):
    client.drop_database(db_name)

# Creating Database and collection in mongodb
db = client[db_name]
countriesPop = db["countriesPopulation"]
citiesPop = db["citiesPopulation"]
worldPop = db["worldPopulation"]
popDensity = db["populationDensity"]
birRate = db["birthRate"]
dthRate = db["deathRate"]
sexRate = db["sexRatio"]


# Function to insert Dataframes into mongodb collections
def insertToDB(df, collection):
    data_dict = df.to_dict("records") # Convert to dictionary
    # removing index from data
    data_dict = [{k: v for k, v in d.items() if k != 'index'} for d in data_dict]
    collection.insert_one({"data":data_dict}) # Insert dict to collection


# Calling function to insert each dataframes into mongoDB collections
insertToDB(df_countries_merged, countriesPop)
insertToDB(df_cityPop, citiesPop)
insertToDB(df_worldPop, worldPop)
insertToDB(df_Density, popDensity)
insertToDB(df_birth, birRate)
insertToDB(df_death, dthRate)
insertToDB(df_sexRatio, sexRate)



print(db.list_collection_names())

['sexRatio', 'populationDensity', 'citiesPopulation', 'worldPopulation', 'birthRate', 'deathRate', 'countriesPopulation']
